In [1]:

import time
import os
import shutil
import pandas as pd
from datetime import datetime
from dateutil import tz
import warnings
import itertools

import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split



In [2]:
# Remove the NaN rows and columns

def removeNaN(curr_feat_1, curr_feat_2):
    curr_feat_1_nan = np.array([])  # numpy.float64
    curr_feat_2_nan = np.array([])  # numpy.float64
    posi = 0
    # *** Check for and remove the Nan rows in from ONLY [curr_feat_2] feature rows
    for posi in range(len(curr_feat_2)):
        if not (math.isnan(curr_feat_2[posi])):# and not(math.isnan(curr_feat_1[posi]))):
            curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
        if (math.isnan(curr_feat_1[posi])):
            # curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            # curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
    return curr_feat_1_nan, curr_feat_2_nan

#### Import .csv dataset into pandas
print('Importing Data Now')

#filename = '/Users/Mandala/PycharmProjects/ML_Project/windowing_results.csv'
filename = 'actrec_windowing_3&4.csv'

# Read data from csv file 
df_feat = pd.read_csv(filename)
df_keys = df_feat.keys()



Importing Data Now


In [3]:
df_feat

,Unnamed: 0,ECG #,start_time,end_time,acc_mean,acc_mean.1,acc_50%,acc_max,acc_eng,acc_var,...,gyro_mean.1,gyro_50%,gyro_max,gyro_eng,gyro_var,gyro_skew,gyro_kurt,gyro_entropy,gyro_mad,Activity
0,svm,3,2019-11-21 09:23:04,2019-11-21 09:23:06,3.074808,3.045923,3.078563,3.116174,21537.502717,0.000125,...,0.125965,1.410155,9.516461,12581.802896,2.238610,2.099189,5.761027,7.729836,1.063948,"(0,0)"
1,svm,3,2019-11-21 09:23:05,2019-11-21 09:23:07,3.070734,3.045923,3.070070,3.109342,21480.392958,0.000086,...,0.047682,1.113528,9.276601,9444.200326,1.857098,2.559431,8.533665,7.729227,0.922118,"(0,0)"
2,svm,3,2019-11-21 09:23:06,2019-11-21 09:23:08,3.070877,3.045923,3.069935,3.100552,21454.066753,0.000077,...,0.037221,1.028702,9.276601,9565.669730,2.037144,2.691735,8.812490,7.727298,0.940215,"(0,0)"
3,svm,3,2019-11-21 09:23:07,2019-11-21 09:23:09,3.072879,3.054522,3.070070,3.102285,21463.128781,0.000059,...,0.037221,1.415488,9.276601,17090.836235,3.175637,1.601641,2.546772,7.727636,1.370081,"(0,0)"
4,svm,3,2019-11-21 09:23:08,2019-11-21 09:23:10,3.073298,3.054522,3.069935,3.102285,21478.429820,0.000060,...,0.037221,1.443367,8.908689,16416.036159,2.970355,1.593336,2.500608,7.726857,1.327227,"(0,0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3901,svm,4,2019-11-21 10:19:30,2019-11-21 10:19:32,11.702805,11.417292,11.703640,11.895382,262961.324078,0.003379,...,0.167190,1.762409,5.488829,9116.831689,0.861082,0.795411,0.229283,7.560080,0.752801,"(2,3)"
3902,svm,4,2019-11-21 10:19:31,2019-11-21 10:19:33,11.695273,11.417292,11.694066,11.895382,263581.173677,0.003774,...,0.163304,1.957578,5.488829,9871.091583,0.889702,0.602416,0.097293,7.563720,0.757811,"(2,3)"
3903,svm,4,2019-11-21 10:19:32,2019-11-21 10:19:34,11.692662,11.417292,11.694293,11.872994,265650.063209,0.003243,...,0.123713,1.751808,5.570260,8338.294151,0.862812,1.014820,1.410757,7.571988,0.706481,"(2,3)"
3904,svm,4,2019-11-21 10:19:33,2019-11-21 10:19:35,11.703943,11.423132,11.703998,11.872994,266023.905128,0.002225,...,0.123713,1.529116,5.570260,6835.576335,0.737167,1.142133,1.965215,7.571474,0.654387,"(2,3)"


In [4]:
print(df_keys)
len(df_keys)

Index(['Unnamed: 0', 'ECG #', 'start_time', 'end_time', 'acc_mean',
       'acc_mean.1', 'acc_50%', 'acc_max', 'acc_eng', 'acc_var', 'acc_skew',
       'acc_kurt', 'acc_entropy', 'acc_mad', 'gyro_mean', 'gyro_mean.1',
       'gyro_50%', 'gyro_max', 'gyro_eng', 'gyro_var', 'gyro_skew',
       'gyro_kurt', 'gyro_entropy', 'gyro_mad', 'Activity'],
      dtype='object')


25

In [5]:
df = df_feat.copy()
df = df.drop(columns=['acc_mean.1', 'gyro_mean.1'])
df.keys()

Index(['Unnamed: 0', 'ECG #', 'start_time', 'end_time', 'acc_mean', 'acc_50%',
       'acc_max', 'acc_eng', 'acc_var', 'acc_skew', 'acc_kurt', 'acc_entropy',
       'acc_mad', 'gyro_mean', 'gyro_50%', 'gyro_max', 'gyro_eng', 'gyro_var',
       'gyro_skew', 'gyro_kurt', 'gyro_entropy', 'gyro_mad', 'Activity'],
      dtype='object')

In [6]:
# Data Preprocessing
df = df_feat.copy()
df = df.dropna()

Xs = df[df_keys[4:24]]
y = df['Activity']

print(Xs)
print(y)

       acc_mean  acc_mean.1    acc_50%    acc_max        acc_eng   acc_var  \
0      3.074808    3.045923   3.078563   3.116174   21537.502717  0.000125   
1      3.070734    3.045923   3.070070   3.109342   21480.392958  0.000086   
2      3.070877    3.045923   3.069935   3.100552   21454.066753  0.000077   
3      3.072879    3.054522   3.070070   3.102285   21463.128781  0.000059   
4      3.073298    3.054522   3.069935   3.102285   21478.429820  0.000060   
...         ...         ...        ...        ...            ...       ...   
3901  11.702805   11.417292  11.703640  11.895382  262961.324078  0.003379   
3902  11.695273   11.417292  11.694066  11.895382  263581.173677  0.003774   
3903  11.692662   11.417292  11.694293  11.872994  265650.063209  0.003243   
3904  11.703943   11.423132  11.703998  11.872994  266023.905128  0.002225   
3905  11.709427   11.502270  11.708944  11.872994  177560.567817  0.001717   

      acc_skew  acc_kurt  acc_entropy   acc_mad  gyro_mean  gyr

In [7]:
# This is the data split

X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=10)

# Training the model
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=7)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [8]:
y_pred = classifier.predict(X_test)
print(y_pred)

['(0,0)' '(2,2)' '(0,3)' '(1,2)' '(2,3)' '(0,3)' '(2,2)' '(1,2)' '(1,2)'
 '(1,3)' '(0,3)' '(2,1)' '(1,2)' '(1,2)' '(1,2)' '(0,3)' '(0,2)' '(1,2)'
 '(0,2)' '(1,1)' '(1,1)' '(0,0)' '(0,3)' '(1,2)' '(0,1)' '(1,2)' '(0,0)'
 '(0,3)' '(1,2)' '(1,2)' '(1,2)' '(1,3)' '(2,1)' '(0,1)' '(2,2)' '(2,2)'
 '(2,1)' '(0,1)' '(0,3)' '(2,2)' '(1,2)' '(1,3)' '(0,0)' '(2,1)' '(2,3)'
 '(2,2)' '(2,3)' '(0,3)' '(1,2)' '(2,2)' '(0,1)' '(2,1)' '(1,2)' '(0,0)'
 '(0,0)' '(1,2)' '(0,1)' '(1,2)' '(0,2)' '(2,2)' '(2,2)' '(0,3)' '(1,2)'
 '(0,2)' '(1,2)' '(2,2)' '(0,1)' '(1,2)' '(0,0)' '(2,1)' '(1,2)' '(0,2)'
 '(1,1)' '(0,3)' '(0,2)' '(0,3)' '(2,2)' '(0,2)' '(2,3)' '(0,0)' '(1,1)'
 '(2,1)' '(0,2)' '(0,3)' '(0,1)' '(0,1)' '(0,2)' '(1,2)' '(1,3)' '(1,3)'
 '(1,2)' '(1,2)' '(2,2)' '(2,3)' '(2,1)' '(1,2)' '(2,1)' '(1,3)' '(0,2)'
 '(1,2)' '(2,1)' '(2,1)' '(1,2)' '(0,3)' '(1,0)' '(2,1)' '(2,3)' '(0,3)'
 '(1,1)' '(0,3)' '(1,2)' '(1,1)' '(1,2)' '(1,3)' '(0,3)' '(2,2)' '(1,2)'
 '(0,1)' '(2,3)' '(2,1)' '(0,2)' '(2,2)' '(0,2)' '(

In [9]:
len(y_pred)

782

In [10]:
len(y_test)

782

In [11]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

[[23  4  3  0  0  1  6  0  0  1  9  0]
 [ 3 17  8  2  0  0  9  0  0  2  4  0]
 [ 3 10 44  2  1  5  8  2  1  7  0  1]
 [ 0  2  4 41  1  4 14  0  0  0  5  1]
 [ 0  0  0  4  1  0  4  1  1  0  0  1]
 [ 1  4  4 15  0 23  8  0  0  1  2  2]
 [ 3  3 16 14  3 11 67  4  1  7 18  5]
 [ 0  3  1  0  0  2  4 41  0  4  6  2]
 [ 0  0  1  2  0  1  3  0  2  0  4  0]
 [ 2  5 12  1  0  0  7  4  0 42  4  3]
 [ 5  1  1  2  0  0 16  3  0  5 42  4]
 [ 0  1  3  2  0  2  9  1  0  2  9 46]]
              precision    recall  f1-score   support

       (0,0)       0.57      0.49      0.53        47
       (0,1)       0.34      0.38      0.36        45
       (0,2)       0.45      0.52      0.49        84
       (0,3)       0.48      0.57      0.52        72
       (1,0)       0.17      0.08      0.11        12
       (1,1)       0.47      0.38      0.42        60
       (1,2)       0.43      0.44      0.44       152
       (1,3)       0.73      0.65      0.69        63
       (2,0)       0.40      0.15      0.22 

In [12]:
df_video = df.loc[(df.Activity == "(0,0)") | (df.Activity == "(0,1)") | (df.Activity == "(0,2)") | (df.Activity == "(0,3)")]
df_video

,Unnamed: 0,ECG #,start_time,end_time,acc_mean,acc_mean.1,acc_50%,acc_max,acc_eng,acc_var,...,gyro_mean.1,gyro_50%,gyro_max,gyro_eng,gyro_var,gyro_skew,gyro_kurt,gyro_entropy,gyro_mad,Activity
0,svm,3,2019-11-21 09:23:04,2019-11-21 09:23:06,3.074808,3.045923,3.078563,3.116174,21537.502717,0.000125,...,0.125965,1.410155,9.516461,1.258180e+04,2.238610,2.099189,5.761027,7.729836,1.063948,"(0,0)"
1,svm,3,2019-11-21 09:23:05,2019-11-21 09:23:07,3.070734,3.045923,3.070070,3.109342,21480.392958,0.000086,...,0.047682,1.113528,9.276601,9.444200e+03,1.857098,2.559431,8.533665,7.729227,0.922118,"(0,0)"
2,svm,3,2019-11-21 09:23:06,2019-11-21 09:23:08,3.070877,3.045923,3.069935,3.100552,21454.066753,0.000077,...,0.037221,1.028702,9.276601,9.565670e+03,2.037144,2.691735,8.812490,7.727298,0.940215,"(0,0)"
3,svm,3,2019-11-21 09:23:07,2019-11-21 09:23:09,3.072879,3.054522,3.070070,3.102285,21463.128781,0.000059,...,0.037221,1.415488,9.276601,1.709084e+04,3.175637,1.601641,2.546772,7.727636,1.370081,"(0,0)"
4,svm,3,2019-11-21 09:23:08,2019-11-21 09:23:10,3.073298,3.054522,3.069935,3.102285,21478.429820,0.000060,...,0.037221,1.443367,8.908689,1.641604e+04,2.970355,1.593336,2.500608,7.726857,1.327227,"(0,0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2844,svm,4,2019-11-21 09:34:11,2019-11-21 09:34:13,12.281716,12.030276,12.281733,12.442244,343468.622873,0.002071,...,0.105477,1.764349,6.280517,1.116108e+04,1.218549,1.084370,1.168706,7.730614,0.834446,"(0,3)"
2845,svm,4,2019-11-21 09:34:12,2019-11-21 09:34:14,12.278336,12.030276,12.282409,12.470821,343433.532963,0.003444,...,0.105477,2.071412,8.059969,2.019756e+04,2.720427,1.105975,0.902809,7.731053,1.301782,"(0,3)"
2846,svm,4,2019-11-21 09:34:13,2019-11-21 09:34:15,12.289080,11.816772,12.286905,12.665642,344956.307420,0.010171,...,0.114072,2.746714,33.635780,1.236166e+05,34.991372,3.414171,11.720577,7.733684,3.218604,"(0,3)"
2847,svm,4,2019-11-21 09:34:14,2019-11-21 09:34:16,12.298408,11.051125,12.280233,13.750516,344464.192462,0.029018,...,0.114696,4.673927,200.702639,1.385629e+06,485.656263,5.199135,32.630925,7.730614,11.108588,"(0,3)"


In [13]:
df_noeval = df.loc[(df.Activity == "(1,0)") | (df.Activity == "(1,1)") | (df.Activity == "(1,2)") | (df.Activity == "(1,3)")]
df_noeval

,Unnamed: 0,ECG #,start_time,end_time,acc_mean,acc_mean.1,acc_50%,acc_max,acc_eng,acc_var,...,gyro_mean.1,gyro_50%,gyro_max,gyro_eng,gyro_var,gyro_skew,gyro_kurt,gyro_entropy,gyro_mad,Activity
820,svm,3,2019-11-21 09:41:18,2019-11-21 09:41:20,1.883530,1.825181,1.874709,1.984105,8063.753190,0.001502,...,0.269321,2.258620,25.299865,1.263743e+05,33.843701,1.974313,2.780591,7.728416,4.222371,"(1,0)"
821,svm,3,2019-11-21 09:41:19,2019-11-21 09:41:21,1.923566,1.825181,1.928421,2.050433,8411.927457,0.003957,...,0.391847,4.274394,25.299865,1.818477e+05,34.187963,1.095897,0.526533,7.727976,4.851755,"(1,0)"
822,svm,3,2019-11-21 09:41:20,2019-11-21 09:41:22,2.001179,1.871113,1.993165,2.133704,9105.637642,0.004811,...,0.391847,10.968888,67.900919,1.108249e+06,224.038542,1.747728,2.623967,7.727365,10.860667,"(1,0)"
823,svm,3,2019-11-21 09:41:21,2019-11-21 09:41:23,2.077061,1.964176,2.084974,2.196352,9799.433129,0.004654,...,0.391847,14.262319,67.900919,1.250822e+06,207.500050,1.619533,2.333681,7.726484,10.524822,"(1,0)"
824,svm,3,2019-11-21 09:41:22,2019-11-21 09:41:24,2.155000,2.029613,2.152064,2.270673,10551.101371,0.004040,...,1.304260,17.543557,67.900919,1.467133e+06,196.764080,1.414512,1.708359,7.726924,10.429836,"(1,0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,svm,4,2019-11-21 09:57:54,2019-11-21 09:57:56,12.482967,11.743825,12.510966,13.545214,292739.086484,0.053673,...,1.306045,12.532116,44.767685,3.526356e+05,41.138338,0.771347,1.469143,7.537224,5.111048,"(1,3)"
3317,svm,4,2019-11-21 09:57:55,2019-11-21 09:57:57,12.528130,11.262910,12.560807,13.099941,296000.157136,0.075265,...,1.306045,11.413925,45.293909,5.015044e+05,93.447178,1.679099,2.456491,7.541683,6.747113,"(1,3)"
3318,svm,4,2019-11-21 09:57:56,2019-11-21 09:57:58,12.198963,11.110727,12.470314,13.099941,279419.879017,0.288925,...,1.647763,10.639799,45.293909,5.174578e+05,102.119428,1.498556,1.710359,7.535830,7.356144,"(1,3)"
3319,svm,4,2019-11-21 09:57:57,2019-11-21 09:57:59,11.851091,11.110727,11.574694,13.099941,264272.638351,0.272041,...,0.238459,5.986880,45.293909,4.332593e+05,122.517538,1.648030,1.802266,7.537963,8.322391,"(1,3)"


In [14]:
df_eval = df.loc[(df.Activity == "(2,0)") | (df.Activity == "(2,1)") | (df.Activity == "(2,2)") | (df.Activity == "(2,3)")]
df_eval

,Unnamed: 0,ECG #,start_time,end_time,acc_mean,acc_mean.1,acc_50%,acc_max,acc_eng,acc_var,...,gyro_mean.1,gyro_50%,gyro_max,gyro_eng,gyro_var,gyro_skew,gyro_kurt,gyro_entropy,gyro_mad,Activity
1697,svm,3,2019-11-21 10:03:01,2019-11-21 10:03:03,1.621534,1.573761,1.627425,1.674689,5188.691635,0.000475,...,0.039863,1.125050,6.662785,9.296909e+03,1.980129,1.393382,1.444033,7.585905,1.100291,"(2,0)"
1698,svm,3,2019-11-21 10:03:02,2019-11-21 10:03:04,1.615623,1.573761,1.617120,1.663221,5135.048204,0.000360,...,0.079724,1.668229,18.281761,3.133922e+04,8.877632,2.834094,9.133880,7.582150,1.910442,"(2,0)"
1699,svm,3,2019-11-21 10:03:03,2019-11-21 10:03:05,1.629216,1.573761,1.618197,1.693909,5204.674267,0.001101,...,0.112834,2.473814,60.852419,9.296145e+05,290.520168,1.058032,-0.296859,7.578578,14.786285,"(2,0)"
1700,svm,3,2019-11-21 10:03:04,2019-11-21 10:03:06,1.663133,1.588408,1.671052,1.722783,5398.586484,0.001077,...,0.135881,14.958366,60.852419,1.121550e+06,243.533771,0.702956,-0.546934,7.576097,13.226526,"(2,0)"
1701,svm,3,2019-11-21 10:03:05,2019-11-21 10:03:07,1.681595,1.644361,1.680816,1.722783,5517.355624,0.000200,...,2.141871,20.806534,67.789943,1.607336e+06,209.459225,0.846116,0.182425,7.576097,11.837218,"(2,0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3901,svm,4,2019-11-21 10:19:30,2019-11-21 10:19:32,11.702805,11.417292,11.703640,11.895382,262961.324078,0.003379,...,0.167190,1.762409,5.488829,9.116832e+03,0.861082,0.795411,0.229283,7.560080,0.752801,"(2,3)"
3902,svm,4,2019-11-21 10:19:31,2019-11-21 10:19:33,11.695273,11.417292,11.694066,11.895382,263581.173677,0.003774,...,0.163304,1.957578,5.488829,9.871092e+03,0.889702,0.602416,0.097293,7.563720,0.757811,"(2,3)"
3903,svm,4,2019-11-21 10:19:32,2019-11-21 10:19:34,11.692662,11.417292,11.694293,11.872994,265650.063209,0.003243,...,0.123713,1.751808,5.570260,8.338294e+03,0.862812,1.014820,1.410757,7.571988,0.706481,"(2,3)"
3904,svm,4,2019-11-21 10:19:33,2019-11-21 10:19:35,11.703943,11.423132,11.703998,11.872994,266023.905128,0.002225,...,0.123713,1.529116,5.570260,6.835576e+03,0.737167,1.142133,1.965215,7.571474,0.654387,"(2,3)"


In [15]:
def kNN(x):
    y = x.copy()
    df = y.dropna()

    Xs = df[df_keys[4:18]]
    y = df['Activity']
    
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=10)

    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors=7)
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    
    from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(accuracy_score(y_test,y_pred))

In [16]:
kNN(df_video)

[[28  5  3  1]
 [ 2 31 17  2]
 [ 8 15 64  4]
 [ 7  4  5 57]]
              precision    recall  f1-score   support

       (0,0)       0.62      0.76      0.68        37
       (0,1)       0.56      0.60      0.58        52
       (0,2)       0.72      0.70      0.71        91
       (0,3)       0.89      0.78      0.83        73

    accuracy                           0.71       253
   macro avg       0.70      0.71      0.70       253
weighted avg       0.72      0.71      0.71       253

0.7114624505928854


In [17]:
kNN(df_noeval)

[[  4   0  10   0]
 [  0  40  16   1]
 [  8  20 107   7]
 [  0   0   4  53]]
              precision    recall  f1-score   support

       (1,0)       0.33      0.29      0.31        14
       (1,1)       0.67      0.70      0.68        57
       (1,2)       0.78      0.75      0.77       142
       (1,3)       0.87      0.93      0.90        57

    accuracy                           0.76       270
   macro avg       0.66      0.67      0.66       270
weighted avg       0.75      0.76      0.75       270

0.7555555555555555


In [18]:
kNN(df_eval)

[[12  3  3  0]
 [ 0 60 11  5]
 [ 2  3 64 13]
 [ 0  6  4 73]]
              precision    recall  f1-score   support

       (2,0)       0.86      0.67      0.75        18
       (2,1)       0.83      0.79      0.81        76
       (2,2)       0.78      0.78      0.78        82
       (2,3)       0.80      0.88      0.84        83

    accuracy                           0.81       259
   macro avg       0.82      0.78      0.80       259
weighted avg       0.81      0.81      0.81       259

0.806949806949807
